In [1]:
path = "/Users/amrtamer/Documents/Internship/"
# path = "/content/drive/MyDrive/Internship/"

In [2]:
import sys
sys.path.append(path + 'Models')
sys.path.append(path + 'Models/Universal/uni2ts/myenv/lib/python3.10/site-packages/')
import utils as ut
import models

/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import torch
from torch.utils.data import DataLoader
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import os
import re
import json
import optuna
import ast
import matplotlib.pyplot as plt
import random
from matplotlib.backends.backend_pdf import PdfPages

In [4]:
data = pd.read_csv(path + r'data/Volve/Volve_cleaned_prepared.csv')
if "Unnamed: 0" in data.columns:
    data = data.drop(columns="Unnamed: 0")
data.head()

,well_name,WELL_BORE_CODE,date,prod_hrs,bhp,bht,dp_tubing,AVG_ANNULUS_PRESS,AVG_CHOKE_SIZE_P,whp,...,linear_regressor,EMA_short,EMA_medium,EMA_long,Rolling_short,Rolling_medium,Rolling_long,Lag_short,Lag_medium,Lag_long
0,7405,NO 15/9-F-1 C,2014-04-22,0.268405,1.029311,0.605973,0.396167,-2.016027,-0.533280,3.281836,...,-0.101892,-0.103579,-0.104768,-0.106929,-0.001355,-0.002618,-0.00519,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,7405,NO 15/9-F-1 C,2014-04-23,0.268405,0.845795,0.634777,0.239818,0.088113,-0.419656,2.849981,...,-0.101892,-0.103579,-0.104768,-0.106929,-0.001355,-0.002618,-0.00519,"[0.0, 0.0, -0.10230979661544379]","[0.0, 0.0, 0.0, 0.0, -0.10230979661544379]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,7405,NO 15/9-F-1 C,2014-04-24,0.268405,0.775025,0.639809,0.199890,-0.688434,-0.402874,2.607708,...,-0.101892,-0.009618,-0.053295,-0.079801,-0.001355,-0.002618,-0.00519,"[0.0, -0.10230979661544379, -0.10230979661544379]","[0.0, 0.0, 0.0, -0.10230979661544379, -0.10230...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.10..."
3,7405,NO 15/9-F-1 C,2014-04-25,0.268405,0.705033,0.642084,0.161493,1.145283,-0.379061,2.364038,...,-0.114759,-0.020781,-0.051611,-0.076565,-0.001355,-0.002618,-0.00519,"[-0.10230979661544379, -0.10230979661544379, 0...","[0.0, 0.0, -0.10230979661544379, -0.1023097966...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.1023097..."
4,7405,NO 15/9-F-1 C,2014-04-26,0.268405,0.625357,0.643889,0.117221,1.145283,-0.340115,2.088747,...,0.082297,-0.004428,-0.037199,-0.066767,-0.009496,-0.002618,-0.00519,"[-0.10230979661544379, 0.1777063666263608, -0....","[0.0, -0.10230979661544379, -0.102309796615443...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.102309796615..."


In [5]:
well_info = dict(set([(j, i) for i, j in data[['WELL_BORE_CODE', 'well_name']].values.tolist()])) # creating a dictionary of well identiification with {code: name} structure
well_info

{5769: 'NO 15/9-F-5 AH',
 7289: 'NO 15/9-F-15 D',
 5599: 'NO 15/9-F-12 H',
 7405: 'NO 15/9-F-1 C',
 7078: 'NO 15/9-F-11 H',
 5351: 'NO 15/9-F-14 H'}

In [6]:
data.columns

Index(['well_name', 'WELL_BORE_CODE', 'date', 'prod_hrs', 'bhp', 'bht',
       'dp_tubing', 'AVG_ANNULUS_PRESS', 'AVG_CHOKE_SIZE_P', 'whp', 'wht',
       'choke_size_percentage', 'oil_vol', 'gas_vol', 'water_vol', 'oil_rate',
       'linear_regressor', 'EMA_short', 'EMA_medium', 'EMA_long',
       'Rolling_short', 'Rolling_medium', 'Rolling_long', 'Lag_short',
       'Lag_medium', 'Lag_long'],
      dtype='object')

In [7]:
identification_column = ['well_name', 'WELL_BORE_CODE', 'date']
single_feature_columns = ['prod_hrs', 'bhp', 'bht', 'dp_tubing', 'AVG_ANNULUS_PRESS', 'AVG_CHOKE_SIZE_P', 'whp', 'wht', 'choke_size_percentage', 'oil_vol', 'gas_vol', 'water_vol', 'linear_regressor', 'EMA_short', 'EMA_medium', 'EMA_long', 'Rolling_short', 'Rolling_medium', 'Rolling_long', 'oil_rate']
multiple_feature_columns = ['Lag_short', 'Lag_medium', 'Lag_long']
# multiple_feature_columns = ['Lag_long']
feature_columns = single_feature_columns + multiple_feature_columns
target_column = ['oil_rate']

In [8]:
data = data[identification_column + feature_columns]
data.shape

(7987, 26)

In [9]:
print(data[['prod_hrs', 'bhp', 'bht', 'dp_tubing', 'AVG_ANNULUS_PRESS', 'AVG_CHOKE_SIZE_P', 'whp', 'wht', 'choke_size_percentage', 'oil_vol', 'gas_vol', 'water_vol', 'linear_regressor', 'EMA_short', 'EMA_medium', 'EMA_long', 'Rolling_short', 'Rolling_medium', 'Rolling_long', 'oil_rate']].mean().round(4))
print(data[['prod_hrs', 'bhp', 'bht', 'dp_tubing', 'AVG_ANNULUS_PRESS', 'AVG_CHOKE_SIZE_P', 'whp', 'wht', 'choke_size_percentage', 'oil_vol', 'gas_vol', 'water_vol', 'linear_regressor', 'EMA_short', 'EMA_medium', 'EMA_long', 'Rolling_short', 'Rolling_medium', 'Rolling_long', 'oil_rate']].std().round(4))

prod_hrs                -0.0
bhp                     -0.0
bht                      0.0
dp_tubing               -0.0
AVG_ANNULUS_PRESS       -0.0
AVG_CHOKE_SIZE_P        -0.0
whp                     -0.0
wht                     -0.0
choke_size_percentage    0.0
oil_vol                  0.0
gas_vol                  0.0
water_vol                0.0
linear_regressor        -0.0
EMA_short                0.0
EMA_medium               0.0
EMA_long                 0.0
Rolling_short           -0.0
Rolling_medium           0.0
Rolling_long            -0.0
oil_rate                 0.0
dtype: float64
prod_hrs                 1.0
bhp                      1.0
bht                      1.0
dp_tubing                1.0
AVG_ANNULUS_PRESS        1.0
AVG_CHOKE_SIZE_P         1.0
whp                      1.0
wht                      1.0
choke_size_percentage    1.0
oil_vol                  1.0
gas_vol                  1.0
water_vol                1.0
linear_regressor         1.0
EMA_short                1.0

In [10]:
data

,well_name,WELL_BORE_CODE,date,prod_hrs,bhp,bht,dp_tubing,AVG_ANNULUS_PRESS,AVG_CHOKE_SIZE_P,whp,...,EMA_short,EMA_medium,EMA_long,Rolling_short,Rolling_medium,Rolling_long,oil_rate,Lag_short,Lag_medium,Lag_long
0,7405,NO 15/9-F-1 C,2014-04-22,0.268405,1.029311,0.605973,0.396167,-2.016027,-0.533280,3.281836,...,-0.103579,-0.104768,-0.106929,-0.001355,-0.002618,-0.005190,-0.102310,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,7405,NO 15/9-F-1 C,2014-04-23,0.268405,0.845795,0.634777,0.239818,0.088113,-0.419656,2.849981,...,-0.103579,-0.104768,-0.106929,-0.001355,-0.002618,-0.005190,-0.102310,"[0.0, 0.0, -0.10230979661544379]","[0.0, 0.0, 0.0, 0.0, -0.10230979661544379]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,7405,NO 15/9-F-1 C,2014-04-24,0.268405,0.775025,0.639809,0.199890,-0.688434,-0.402874,2.607708,...,-0.009618,-0.053295,-0.079801,-0.001355,-0.002618,-0.005190,0.177706,"[0.0, -0.10230979661544379, -0.10230979661544379]","[0.0, 0.0, 0.0, -0.10230979661544379, -0.10230...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.10..."
3,7405,NO 15/9-F-1 C,2014-04-25,0.268405,0.705033,0.642084,0.161493,1.145283,-0.379061,2.364038,...,-0.020781,-0.051611,-0.076565,-0.001355,-0.002618,-0.005190,-0.042238,"[-0.10230979661544379, -0.10230979661544379, 0...","[0.0, 0.0, -0.10230979661544379, -0.1023097966...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.1023097..."
4,7405,NO 15/9-F-1 C,2014-04-26,0.268405,0.625357,0.643889,0.117221,1.145283,-0.340115,2.088747,...,-0.004428,-0.037199,-0.066767,-0.009496,-0.002618,-0.005190,0.028674,"[-0.10230979661544379, 0.1777063666263608, -0....","[0.0, -0.10230979661544379, -0.102309796615443...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.102309796615..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7982,5769,NO 15/9-F-5 AH,2016-08-22,0.268405,-0.880933,-0.806036,-0.814175,1.149266,1.150153,-1.140330,...,-0.692802,-0.702316,-0.715245,-0.692202,-0.703021,-0.724022,-0.683109,"[-0.6852415797773639, -0.6943429268335604, -0....","[-0.6933422169244923, -0.687878486910018, -0.6...","[-0.7133929373653101, -0.7128670168291575, -0...."
7983,5769,NO 15/9-F-5 AH,2016-08-23,0.268405,-0.880933,-0.806036,-0.814175,1.158798,1.150153,-1.136215,...,-0.689514,-0.699464,-0.713067,-0.690182,-0.699482,-0.722275,-0.677835,"[-0.6943429268335604, -0.6840874763785847, -0....","[-0.687878486910018, -0.6852415797773639, -0.6...","[-0.7128670168291575, -0.704496114962062, -0.6..."
7984,5769,NO 15/9-F-5 AH,2016-08-24,0.268405,-0.880933,-0.806036,-0.814175,1.141085,1.150153,-1.137527,...,-0.687737,-0.697357,-0.711216,-0.688940,-0.696914,-0.720063,-0.679069,"[-0.6840874763785847, -0.6831086798251896, -0....","[-0.6852415797773639, -0.6943429268335604, -0....","[-0.704496114962062, -0.6974619277910209, -0.6..."
7985,5769,NO 15/9-F-5 AH,2016-08-25,0.268405,-1.334973,-1.285316,-1.327391,1.140689,1.150153,-1.140776,...,-0.686410,-0.695555,-0.709500,-0.685783,-0.695013,-0.718130,-0.678646,"[-0.6831086798251896, -0.6778348655598816, -0....","[-0.6943429268335604, -0.6840874763785847, -0....","[-0.6974619277910209, -0.6908660077334403, -0...."


In [11]:
for column in multiple_feature_columns:
    try:
        data[column] = data[column].apply(ast.literal_eval)
    except Exception:
        pass

In [12]:
data.isna().any()

well_name                False
WELL_BORE_CODE           False
date                     False
prod_hrs                 False
bhp                      False
bht                      False
dp_tubing                False
AVG_ANNULUS_PRESS        False
AVG_CHOKE_SIZE_P         False
whp                      False
wht                      False
choke_size_percentage    False
oil_vol                  False
gas_vol                  False
water_vol                False
linear_regressor         False
EMA_short                False
EMA_medium               False
EMA_long                 False
Rolling_short            False
Rolling_medium           False
Rolling_long             False
oil_rate                 False
Lag_short                False
Lag_medium               False
Lag_long                 False
dtype: bool

In [13]:
# Get the numerical index of the column 'oil_rate'
column_name = 'oil_rate'
column_index = data[feature_columns].columns.get_loc(column_name)
column_index

19

In [14]:
batch_size = 96
days_window = 5
predictions_days = 14
training_gap = predictions_days + days_window

In [15]:
# sequences = []
# for well_name in data['well_name'].unique():
#     well_data = data[data['well_name'] == well_name].sort_values(by='date').reset_index()
#     for i in range(len(well_data) - training_gap + 1):
#         single_features = np.array(well_data.iloc[i:i + days_window][single_feature_columns])
#         multiple_features = np.array(well_data.iloc[i:i + days_window][multiple_feature_columns])
#         multiple_flattened = []
#         for k in multiple_features:
#            flattened_list = [item for sublist in k for item in sublist]
#            multiple_flattened.append(flattened_list)
#         multiple_flattened = np.array(multiple_flattened)
#         inputs = np.concatenate((single_features, multiple_flattened), axis=1)
#         targets = well_data.iloc[i + days_window:i + days_window + predictions_days][target_column].values
#         inputs_dates = list(well_data.iloc[i:i + days_window]['date'])
#         targets_dates = list(well_data.iloc[i + days_window:i + days_window + predictions_days]['date'])
#         inputs_index = list(well_data.iloc[i:i + days_window]['index'])
#         targets_index = list(well_data.iloc[i + days_window:i + days_window + predictions_days]['index'])
#         sequences.append((well_name, inputs_dates, targets_dates, inputs_index, targets_index, inputs, targets))
#         if len(well_data.iloc[i:i + days_window + predictions_days]['well_name'].unique()) > 1:
#           print('violation')

# dataset = pd.DataFrame(sequences, columns=['well_name', 'inputs_dates', 'targets_dates', 'inputs_index', 'targets_index', 'inputs', 'targets', ])
# dataset.to_pickle(path + "/data/Volve/all_data.pkl")
# dataset.to_csv(path + "/data/Volve/all_data.csv")
# dataset

In [16]:
train_size = 0.6
val_size = 0.2
test_size = 0.2

In [17]:
test_size_param1 = test_size
test_size_param2 = val_size / (1 - test_size_param1)

In [18]:
# For shuffled train/val

# dataset = pd.read_pickle(path + "/data/Volve/all_data.pkl")
# train_wells = []
# test_wells = []
# for i in well_info.keys():
#     well = dataset[dataset['well_name'] == i]
#     train_set_length = int((train_size + val_size) * len(well))
#     train_wells.append(well[:train_set_length + 1])
#     test_set_length = int(test_size * len(well))
#     test_wells.append(well[-test_set_length:])

# # Flatten the lists of DataFrames
# temp_data = pd.concat(train_wells).reset_index(drop=True)
# data_test = pd.concat(test_wells).reset_index(drop=True)


# data_train, data_val = train_test_split(temp_data, test_size=test_size_param2, stratify=temp_data['well_name'], shuffle=True)

# data_train.reset_index(drop=True, inplace=True)
# data_val.reset_index(drop=True, inplace=True)
# data_test.reset_index(drop=True, inplace=True)

# data_train.to_pickle(path + "/data/Volve/data_train.pkl")
# data_val.to_pickle(path + "/data/Volve/data_val.pkl")
# data_test.to_pickle(path + "/data/Volve/data_test.pkl")

# data_train.to_csv(path + "/data/Volve/data_train.csv")
# data_val.to_csv(path + "/data/Volve/data_val.csv")
# data_test.to_csv(path + "/data/Volve/data_test.csv")


In [19]:
# # For in order train/val

# all_data = pd.read_pickle(path + "/data/Volve/all_data.pkl")

# # Initialize lists to store DataFrames for each well
# train_wells = []
# val_wells = []
# test_wells = []

# # Iterate over each well and split the data
# for well_name in well_info.keys():
#     well = all_data[all_data['well_name'] == well_name]
#     train_set_length = int(train_size * len(well))
#     val_set_length = int(val_size * len(well))
#     test_set_length = int(test_size * len(well))
    
#     train_wells.append(well[:train_set_length])
#     val_wells.append(well[train_set_length:train_set_length + val_set_length])
#     test_wells.append(well[-test_set_length:])

# # Flatten the lists of DataFrames
# data_train = pd.concat(train_wells).reset_index(drop=True)
# data_val = pd.concat(val_wells).reset_index(drop=True)
# data_test = pd.concat(test_wells).reset_index(drop=True)

In [20]:
all_data = pd.read_pickle(path + "data/Volve/all_data.pkl")
data_train = pd.read_pickle(path + "data/Volve/data_train.pkl")
data_val = pd.read_pickle(path + "data/Volve/data_val.pkl")
data_test = pd.read_pickle(path + "data/Volve/data_test.pkl")

In [21]:
# all_data = pd.read_csv(path + "data/Volve/all_data.csv")
# data_train = pd.read_csv(path + "data/Volve/data_train.csv")
# data_val = pd.read_csv(path + "data/Volve/data_val.csv")
# data_test = pd.read_csv(path + "data/Volve/data_test.csv")

In [22]:
# for df in [all_data, data_train, data_val, data_test]:
#     # Ensure each entry in 'inputs' and 'targets' is a NumPy array
#     df['inputs'] = df['inputs'].apply(lambda x: np.array(x, dtype=np.float32))
#     df['targets'] = df['targets'].apply(lambda x: np.array(x, dtype=np.float32))

In [23]:
data_train.shape, data_val.shape, data_test.shape

((4730, 7), (1577, 7), (1573, 7))

In [24]:
check = True
for i, j in well_info.items():
    if len(all_data[all_data['well_name'] == i]) < 10:
        print("The input/output gap is too high for well", i)
        check = False
if check:
    print("The input/output gap is fine")

The input/output gap is fine


In [25]:
check=True
for i in [data_train, data_val, data_test]:
    for j in well_info.keys():
        if j not in set(list(i['well_name'])):
            print("Some datasets do not have specific well")
            check = False
            break
if check:
    print("All datasets have all wells")

All datasets have all wells


In [26]:
for df in [data_train, data_val, data_test]:
    for column in df.columns:
        try:
            data[column] = data[column].apply(ast.literal_eval)
        except Exception:
            pass

In [27]:
data_train['inputs'][0].shape, data_train['targets'][0].shape

((5, 38), (14, 1))

In [28]:
if torch.backends.mps.is_available():
    device = 'mps'
elif torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu' 
device

'mps'

In [29]:
eval_interval = 500
test_eval_interval = 500
test_iterations = 7500
warmup_steps = 6000 # hyper-parameters for hyperprameter finetuning

In [30]:
train_dataset = ut.TimeSeriesDataset(data_train)
val_dataset = ut.TimeSeriesDataset(data_val)
test_dataset = ut.TimeSeriesDataset(data_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [31]:
visualization_test_dataset = ut.VisualizationTimeSeriesDataset(data_test)

visualization_test_loader = DataLoader(visualization_test_dataset, batch_size=int(len(test_dataset) / 50), shuffle=True)

In [32]:
input_length = data_train['inputs'][0].shape[1]
output_length = predictions_days

In [33]:
# Define an objective function for Optuna
def objective(trial):
    vector_length = trial.suggest_int('vector_length', 16, 2048, step=16)
    num_layers = trial.suggest_int('num_layers', 1, 5) # maybe increase it to 6
    dropout = trial.suggest_float('dropout', 0.1, 0.6)
    lr = trial.suggest_float('lr', 1e-5, 1e-2, log=True)
    gamma = trial.suggest_float('gamma', 0.95, 0.99)
    model = models.LSTM(input_length=input_length, hidden_length=vector_length, output_length=output_length, num_layers=num_layers, dropout=dropout).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=gamma) # setting optimizer scheduler
    train_loader_iter = iter(train_loader)
    # Training loop
    for i in range(test_iterations):
        model.train()
        if (i % test_eval_interval == 0 or i == test_iterations - 1):
            losses = ut.estimate_loss(model, train_loader, val_loader)
            trial.report(losses['val'], i)
            if i != 0:
                scheduler.step()
            # if i >= warmup_steps and trial.should_prune():
            #     raise optuna.TrialPruned()
        try:
            X_batch, Y_batch = next(train_loader_iter)
        except StopIteration:
            train_loader_iter = iter(train_loader)
            X_batch, Y_batch = next(train_loader_iter)
            
        X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
        logits, loss = model(X_batch, Y_batch)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()
    val_metrics = ut.evaluate('val', model)
    return val_metrics['Loss']

def load_model(name=""): 
    config_path = path + r'Models/Global/LSTM/Model/Complete/'
    if name != "":
        name = f'{name}_'
    model = torch.load(config_path + f'{name}complete_params.pth')
    optimizer = torch.load(config_path + f'{name}complete_optimizer.pth')
    scheduler = torch.load(config_path + f'{name}complete_scheduler.pth')
    return model, optimizer, scheduler



In [34]:
# # Create a study and optimize the objective function
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=200)

# # Get the best hyperparameters
# best_params = study.best_params
# print('Best hyperparameters:', best_params)

# # Train the final model with the best hyperparameters
# vector_length = best_params['vector_length']
# num_layers = best_params['num_layers']
# dropout = best_params['dropout']
# lr = best_params['lr']
# gamma = best_params['gamma']

In [35]:
vector_length = 1024
num_layers = 3
dropout = 0.19057012152026273
lr = 0.0001911914637460164
gamma = 0.9806898435934219 # 2nd best results on val set full training

In [34]:
vector_length = 336
num_layers = 3
dropout = 0.12831089625320793
lr = 0.0003724263259631224
gamma = 0.978770138138513 # best results on val set full training

In [35]:
# Example usage
context_size = days_window - 1
X_batch, Y_batch = next(iter(train_loader))
X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
print(f'The shape of input is: {X_batch.shape} and the shape of the output is {Y_batch.shape}')
model = models.LSTM(input_length=input_length, hidden_length=vector_length, output_length=output_length, num_layers=num_layers, dropout=dropout).to(device)
output, loss = model(X_batch, Y_batch)
print(f'The output shape of the LSTM will be {tuple(output.shape)} and a an example initial loss is {loss}')

The shape of input is: torch.Size([96, 5, 38]) and the shape of the output is torch.Size([96, 14])
The output shape of the LSTM will be (96, 14) and a an example initial loss is 1.2377169132232666


In [40]:
model = models.LSTM(input_length=input_length, hidden_length=vector_length, output_length=output_length, num_layers=num_layers, dropout=dropout).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=gamma) # setting optimizer scheduler

In [41]:
iterations = 30000

In [42]:
train_loader_iter = iter(train_loader)
log_file = ut.get_next_log_file(path + "/Models/Global/LSTM/Logs")
config_directory = r'Models/Global/LSTM/Model/State/'
model_directory = r'Models/Global/LSTM/Model/Complete/'

log_message = f"Hyperparameters: Vector Length = {vector_length}, Number of Layers = {num_layers}, Dropout = {dropout}, Learning Rate = {lr}, Scheduler Gamma = {gamma}\n"

ut.log_to_file(log_file, log_message)
model.train()
try:
    for i in range(iterations):
        model.train()  # Set model to training mode
        if i % eval_interval == 0 or i == iterations - 1:
            # Evaluate and save model parameters periodically
            model.save_params(optimizer=optimizer, scheduler=scheduler, config_path=path+config_directory, model_path=path+model_directory, name='current')  # Saving parameters
            model.eval()
            losses = ut.estimate_loss(model, train_loader, val_loader, device=device)  # Estimate train and val loss
            model.train()
            log_message = f"step {i}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}"
            print(log_message)
            ut.log_to_file(log_file, log_message)  # Log to file
            if i != 0:
                scheduler.step()  # Perform scheduler step
        try:
            # Get the next batch
            X_batch, Y_batch = next(train_loader_iter)
        except StopIteration:
            # Reinitialize the iterator if the DataLoader is exhausted
            train_loader_iter = iter(train_loader)
            X_batch, Y_batch = next(train_loader_iter)

        X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
        logits, loss = model(X_batch, Y_batch)  # Run inputs and expected outputs through model
        optimizer.zero_grad(set_to_none=True)  # Set gradients to 0 (to ensure gradients don't explode)
        loss.backward()  # Perform backpropagation
        optimizer.step()  # Perform parameter adjustment

except KeyboardInterrupt:
    print("Training ended manually by user.\n")
model.eval()
print("Training is complete")


step 0: train loss 1.1340, val loss 1.1149
step 500: train loss 0.0551, val loss 0.0543
step 1000: train loss 0.0487, val loss 0.0507
step 1500: train loss 0.0446, val loss 0.0487
step 2000: train loss 0.0369, val loss 0.0428
step 2500: train loss 0.0348, val loss 0.0418
step 3000: train loss 0.0299, val loss 0.0430
step 3500: train loss 0.0256, val loss 0.0367
step 4000: train loss 0.0210, val loss 0.0316
step 4500: train loss 0.0176, val loss 0.0294
step 5000: train loss 0.0162, val loss 0.0289
step 5500: train loss 0.0139, val loss 0.0272
step 6000: train loss 0.0137, val loss 0.0266
step 6500: train loss 0.0122, val loss 0.0250
step 7000: train loss 0.0123, val loss 0.0257
step 7500: train loss 0.0114, val loss 0.0264
step 8000: train loss 0.0099, val loss 0.0241
step 8500: train loss 0.0105, val loss 0.0245
step 9000: train loss 0.0086, val loss 0.0230
step 9500: train loss 0.0082, val loss 0.0241
step 10000: train loss 0.0083, val loss 0.0231
step 10500: train loss 0.0075, val lo

In [43]:
model.eval()
train_metrics = ut.evaluate(model, train_loader, device=device)
val_metrics = ut.evaluate(model, val_loader, device=device)
test_metrics = ut.evaluate(model, test_loader, device=device)
ut.log_to_file(log_file, str(f"\nMetrics = {test_metrics}"))

Metrics on dataset:

Loss is 0.0050
MAE is 0.0430
RMSE is 0.0704
R² is 0.9955


Metrics on dataset:

Loss is 0.0212
MAE is 0.0749
RMSE is 0.1472
R² is 0.9799


Metrics on dataset:

Loss is 0.0116
MAE is 0.0765
RMSE is 0.1060
R² is 0.5306




In [70]:
label_model_files(old_name='current', new_name='best')

Files have been renamed successfully.


In [79]:
model, optimizer, scheduler = load_model('best')

In [46]:
ut.produce_graphs(model, visualization_test_loader, data, data_test, days_window, predictions_days, path + "Models/Global/XGBoost/Graphs", well_info, test_size, column_index=column_index, device=device)
test_metrics = ut.evaluate(model, test_loader, device=device)

Metrics on dataset:

Loss is 0.0116
MAE is 0.0765
RMSE is 0.1060
R² is 0.5306




1. consecutive set splits (bad)
2. experiment with the lags (slightly worse on test set)
3. add more to chronos
4. change mean and std to training data

universal:
1. try to find tutorials